In [ ]:
# event = {"calamity": {"famine", "plague", "tidal_wave", "tsunami" , "Avalanche", "Blizzard", "Earthquake", "Fire", "Flood", "Heat wave", "Hurricane", "Landslide", "Lightning strike", "Volcanic eruption"}}

In [2]:
def get_event_typing(w, thres = 0.0, m = 3, depth = 3):
    hyper = lambda s: s.hypernyms()
    ws = wn.synsets(w)
    types = {}
    for e in ws:
        tmp_event_types = list(e.closure(hyper))
        t = 0
        c = 0 
        i = 0
        tmp_types = ""
        for tmp_event in tmp_event_types:
            d = wn.path_similarity(e, tmp_event)
    #         print(tmp_event, d)
            b = m**i
            t = t + d/b
            if d and (d>=thres/b):
                tmp_types = tmp_types + ":" + tmp_event.name().split(".")[0]
                c = c + 1
                if c >= depth:
                    break
            i = i + 1
        if len(tmp_types) > 0:
            types[t] = tmp_types
    #     print("\n")


    types = {k: v for k, v in sorted(types.items(), key=lambda item: item[0], reverse=True)}
    
    return types

In [4]:
import os
import json

wf = json.load(open("word_fequency.json"))
wfo = {}
c = 0
for t in wf:
    c = c + wf[t]

for t in wf:
    wfo[t] = wf[t]/c
    
    
json_object = json.dumps(wfo)

with open("word_fequency_norm.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
text = '''Ukrainian President Volodymyr Zelensky announced Tuesday on Telegram that Viktor Medvedchuk, a pro-Russian Ukrainian politician and oligarch, had been detained in a "special operation."

Zelensky posted a photo of a handcuffed and disheveled-looking Medvedchuk wearing fatigues, with the caption, "A special operation was carried out thanks to the SBU [the Security Service of Ukraine]. Well done! Details later."
Prior to Russia's invasion of Ukraine, Medvedchuk had faced allegations of treason in Ukraine and had been under house arrest. His whereabouts had been unknown in the weeks following the invasion. Some observers speculated that Medvedchuk or one of his allies might be the Kremlin's preference to lead a puppet government in Ukraine if the Feb. 24 invasion succeeded in toppling Zelensky.

Medvedchuk was sanctioned by the US in 2014 "for threatening the peace, security, stability, sovereignty, or territorial integrity of Ukraine, and for undermining Ukraine’s democratic institutions and processes."

But the wealthy businessman also served as a go-between for Moscow and Kyiv after the outbreak of the Donbas conflict in 2014 by leveraging his personal ties with Putin. In a 2019 interview with filmmaker Oliver Stone, Putin acknowledged that he was godfather to Medvedchuk's daughter.

"I would not say that we are very close but we know each other well," Putin said. "He was [former Ukrainian] President [Leonid] Kuchma’s chief of staff, and it was in this capacity at the time that he asked me to take part in the christening of his daughter. According to Russian Orthodox tradition, you can't refuse such a request."

Medvedchuk also had notoriety in Ukraine for his role as the Soviet state-appointed defense attorney for the Ukrainian dissident poet Vasyl Stus, who died in a Soviet labor camp in 1985.

In a statement, SBU head Ivan Bakanov said, "You may be a pro-Russian politician and work for the aggressor state for years. You may hide from justice lately. You may even wear a Ukrainian military uniform for camouflage… But will it help you to escape punishment? Not at all! Shackles are waiting for you. And for the same traitors of Ukraine as you!"

Bakanov added, "Pro-Russian traitors and agents of the Russian intelligence services, remember — your crimes have no statute of limitations. And there are no hiding places where we wouldn’t find you!"

CNN was not immediately able to reach a legal representative for Medvedchuk.


'''

In [26]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from time import time
# nltk.download('words')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')


wfo = json.load(open("word_fequency_NORM.json"))

text = "The eruption of an underwater volcano near Tonga on Saturday was likely the biggest recorded anywhere on the planet in more than 30 years, according to experts. Dramatic images from space captured the eruption in real time, as a huge plume of ash, gas and steam was spewed up to 20 kilometers into the atmosphere and tsunami waves were sent crashing across the Pacific."
# text = "At least seven historically Black colleges and universities (HBCUs) across the United States received back-to-back bomb threats this week, forcing students to evacuate or shelter in place while authorities investigated. The threats come amid a dramatic rise in bombings in the US and follow bomb threats at other US colleges last November."
# text = "Crews fighting a massive fire along the central coast of California near the iconic Highway 1 made progress Sunday in containing the blaze, but dozens of homes remained under evacuation orders. The Colorado fire ignited Friday evening in Palo Colorado Canyon in the Big Sur region of Monterey County and swelled to 1050 acres Saturday, up from 100 acres a day prior."

text = re.sub('\n+',' ', text)
text = re.sub('\t+',' ', text)
text = re.sub('\s+',' ', text)

number_of_sentences = len(sent_tokenize(text))


# print(text)
print("Number of Sentences: ", number_of_sentences)
print("Number of Tokens: ", len(word_tokenize(text)))
print("Number of Characters: ", len(text))

print("\n")


verb_max = 0.001
noun_max = 0.001

start_time = time()
pos = nltk.tag.pos_tag(text.split())


for p in pos:
    event_types = []
    w = p[0].lower()
    ps = p[1]
    if ps =="NNS":
        continue
    if ((not ps=="NN") and (not ps.startswith("VB"))) or ((not (p[0] in words.words())) or (not (p[0] in wf))):
        tmp_w = WordNetLemmatizer().lemmatize(w,'v')
        if tmp_w and (tmp_w is not w):
            w = tmp_w
            ps = nltk.tag.pos_tag(w.split())[0][1]
        else:
            tmp_w = WordNetLemmatizer().lemmatize(w,'n')
            if tmp_w and (tmp_w is not w):
                w = tmp_w
                ps = nltk.tag.pos_tag(w.split())[0][1]
    if ps=="NN" or ps.startswith("VB"):
        if w in words.words() and w in wfo:
            if ps=="NN" and wfo[w] > noun_max:
                continue
            if ps.startswith("VB") and wfo[w] > verb_max:
                continue
            if p[0].lower() in wfo:
                if ps=="NN" and wfo[p[0].lower()] > noun_max:
                    continue
                if ps.startswith("VB") and wfo[p[0].lower()] > verb_max:
                    continue
            event_types_dict = get_event_typing(w, thres = 0.0, m = 3, depth=3)
            event_types = [elem for elem in event_types_dict.values()]
            if len(event_types)==0:
                f = fn.frames(p[0].lower())
                if len(f) > 0:
                    event_types = [t["name"] for t in f]
                else:
                    f = fn.frames_by_lemma(p[0].lower())
                    if len(f) > 0:
                        event_types = [t["name"] for t in f]
                    else:
                        f = fn.frames(w)
                        if len(f) > 0:
                            event_types = [t["name"] for t in f]
                        else:
                            f = fn.frames_by_lemma(w)
                            if len(f) > 0:
                                event_types = [t["name"] for t in f]
            print(p[0], event_types, wfo[w], ps, w)
#                 print(p[0])
#                 print(w)


total_time = time() - start_time
avg_time = total_time/number_of_sentences
print("\n")
print("Total Time: ", total_time)
print("Average Time per Sentence: ", avg_time)
                

Number of Sentences:  2
Number of Tokens:  69
Number of Characters:  369


eruption [':emergence:beginning:happening'] 1.2106156464808614e-06 NN eruption
volcano [':mountain:natural_elevation:geological_formation'] 1.2106156464808614e-06 NN volcano
recorded [':indicate:inform:communicate', ':remind'] 0.0004951417994106723 NN record
planet [':celestial_body:natural_object:whole'] 1.4527387757770337e-05 NN planet
according [':give:transfer'] 0.0005108798028149235 VBG according
space [':location:space:object', ':type:block:artifact', ':put:move'] 8.111124831421772e-05 NN space
captured [':represent:re-create:make', ':assume:take', ':get'] 1.4527387757770337e-05 NN capture
eruption [':emergence:beginning:happening'] 1.2106156464808614e-06 NN eruption
gas [':fuel:hydrocarbon:substance', ':overstate:misinform:inform', ':attack:contend'] 8.353247960717944e-05 NN gas
steam [':cook:change_integrity:change', ':clean:change', ':emit'] 1.815923469721292e-05 NN steam
atmosphere [':quality:attribute